In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from scipy.stats import poisson
import xgboost as xgb
import torch



# CONFIG 
DATA_PATH = r"data2_filled_safe.csv"
SPLIT_DATE = pd.to_datetime("2025-11-01")
INITIAL_ELO = 1500
N_SIM = 5000
MAX_GOALS = 10
CV_FOLDS = 5
SEED = 42
np.random.seed(SEED)

#  LOAD DATA 
df = pd.read_csv(DATA_PATH)
df['match_date'] = pd.to_datetime(df['match_date'], errors='coerce')
df = df.sort_values('match_date').reset_index(drop=True)

#  Keep original team names 
df['home_team_name_orig'] = df['home_team']
df['away_team_name_orig'] = df['away_team']

#  Encode categorical features for modeling 
for col in ['home_team', 'away_team']:
    df[col] = df[col].astype('category').cat.codes

#  Feature Engineering 
df['match_id'] = np.arange(len(df))
if 'home_elo' not in df.columns or df['home_elo'].isnull().all():
    df['home_elo'] = INITIAL_ELO
    df['away_elo'] = INITIAL_ELO
df['match_num_home'] = df.groupby('home_team').cumcount() + 1
df['match_num_away'] = df.groupby('away_team').cumcount() + 1

#  Features 
feature_cols = df.select_dtypes(include=np.number).columns.tolist()
feature_cols = [c for c in feature_cols if c not in ['home_goals','away_goals']]

train_df = df[df['match_date'] < SPLIT_DATE].copy()
test_df = df[df['match_date'] >= SPLIT_DATE].copy()
y_train_home = train_df['home_goals']
y_train_away = train_df['away_goals']

#  Impute 
imp = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imp.fit_transform(train_df[feature_cols]), columns=feature_cols)
X_test = pd.DataFrame(imp.transform(test_df[feature_cols]), columns=feature_cols)

#  XGBoost Poisson parameters 
xgb_params = {
    "objective": "count:poisson",
    "learning_rate": 0.02,
    "max_depth": 4,
    "subsample": 0.85,
    "colsample_bytree": 0.8,
    "reg_lambda": 1.2,
    "seed": SEED,
    "tree_method": "hist",
    "verbosity": 0
}

#  CV with XGBoost 
tscv = TimeSeriesSplit(n_splits=CV_FOLDS)
oof_preds = np.zeros((len(train_df), 2))
oof_probs = np.zeros((len(train_df), 3))
oof_true_outcome = np.full(len(train_df), -1, dtype=int)
cv_metrics = []

for fold, (train_idx, val_idx) in enumerate(tscv.split(X_train), 1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr_home, y_val_home = y_train_home.iloc[train_idx], y_train_home.iloc[val_idx]
    y_tr_away, y_val_away = y_train_away.iloc[train_idx], y_train_away.iloc[val_idx]

    dtrain_home = xgb.DMatrix(X_tr, label=y_tr_home)
    dval_home = xgb.DMatrix(X_val, label=y_val_home)
    dtrain_away = xgb.DMatrix(X_tr, label=y_tr_away)
    dval_away = xgb.DMatrix(X_val, label=y_val_away)

    evals_home = [(dtrain_home, 'train'), (dval_home, 'eval')]
    evals_away = [(dtrain_away, 'train'), (dval_away, 'eval')]

    home_model = xgb.train(xgb_params, dtrain_home, num_boost_round=2000, evals=evals_home,
                           early_stopping_rounds=50, verbose_eval=False)
    away_model = xgb.train(xgb_params, dtrain_away, num_boost_round=2000, evals=evals_away,
                           early_stopping_rounds=50, verbose_eval=False)

    pred_val_home = home_model.predict(dval_home).clip(0.05, 10)
    pred_val_away = away_model.predict(dval_away).clip(0.05, 10)
    oof_preds[val_idx, 0] = pred_val_home
    oof_preds[val_idx, 1] = pred_val_away

    for i, idx in enumerate(val_idx):
        ph = poisson.pmf(np.arange(MAX_GOALS+1), pred_val_home[i])
        pa = poisson.pmf(np.arange(MAX_GOALS+1), pred_val_away[i])
        joint = np.outer(ph, pa)
        oof_probs[idx] = [np.tril(joint, -1).sum(), np.trace(joint), np.triu(joint, 1).sum()]

    h_goals = y_val_home.values
    a_goals = y_val_away.values
    oof_true_outcome[val_idx] = np.where(h_goals>a_goals,0,np.where(h_goals==a_goals,1,2))

    rmse_home = np.sqrt(np.mean((h_goals - pred_val_home)**2))
    rmse_away = np.sqrt(np.mean((a_goals - pred_val_away)**2))
    r2_home = r2_score(h_goals, pred_val_home)
    r2_away = r2_score(a_goals, pred_val_away)
    match_acc = np.mean(np.sign(h_goals - a_goals) == np.sign(pred_val_home - pred_val_away))
    cv_metrics.append(((rmse_home+rmse_away)/2,(r2_home+r2_away)/2,match_acc))
    print(f"Fold {fold} → RMSE: {(rmse_home+rmse_away)/2:.3f}, R²: {(r2_home+r2_away)/2:.3f}, MatchAcc: {match_acc*100:.2f}%")

cv_rmse = np.mean([m[0] for m in cv_metrics])
cv_r2 = np.mean([m[1] for m in cv_metrics])
cv_acc = np.mean([m[2] for m in cv_metrics])
print(f"\nAverage CV → RMSE: {cv_rmse:.3f}, R²: {cv_r2:.3f}, Accuracy: {cv_acc*100:.2f}%")

#  Calibration 
has_true = oof_true_outcome >= 0
if has_true.sum() >= 50:
    calibrator = LogisticRegression(max_iter=2000, solver='lbfgs', random_state=SEED)
    calibrator.fit(oof_probs[has_true], oof_true_outcome[has_true])
    print(" Calibrator trained.")
else:
    calibrator = None
    print(" Not enough OOF data for calibration.")

#  Final XGBoost models 
dtrain_home_full = xgb.DMatrix(X_train, label=y_train_home)
dtrain_away_full = xgb.DMatrix(X_train, label=y_train_away)
home_final = xgb.train(xgb_params, dtrain_home_full, num_boost_round=2000,
                       evals=[(dtrain_home_full,'train')], early_stopping_rounds=50, verbose_eval=False)
away_final = xgb.train(xgb_params, dtrain_away_full, num_boost_round=2000,
                       evals=[(dtrain_away_full,'train')], early_stopping_rounds=50, verbose_eval=False)
feature_cols = X_train.columns.tolist()
joblib.dump(feature_cols, "feature_cols.pkl")

joblib.dump(home_final, "home_xgb_poisson.pkl")
joblib.dump(away_final, "away_xgb_poisson.pkl")
print(" Final models saved.")

#  Predict test set 
dtest = xgb.DMatrix(X_test)
y_pred_test_home = home_final.predict(dtest).clip(0.05,10)
y_pred_test_away = away_final.predict(dtest).clip(0.05,10)



#  Monte Carlo simulation 
homes = np.random.poisson(y_pred_test_home[:,None], (len(y_pred_test_home), N_SIM))
aways = np.random.poisson(y_pred_test_away[:,None], (len(y_pred_test_away), N_SIM))
sim_results = test_df[['match_date','home_team_name_orig','away_team_name_orig']].copy()
sim_results['exp_home_goals'] = homes.mean(axis=1)
sim_results['exp_away_goals'] = aways.mean(axis=1)
sim_results['prob_home_win'] = (homes>aways).mean(axis=1)
sim_results['prob_draw'] = (homes==aways).mean(axis=1)
sim_results['prob_away_win'] = (homes<aways).mean(axis=1)
sim_results = sim_results.rename(columns={'home_team_name_orig':'home_team_name','away_team_name_orig':'away_team_name'})
sim_results.to_csv("simulated_matches_with_probs.csv", index=False)
print("✅ Simulation saved.")

#  Season simulation 
N_SEASON_SIM = 10000 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(42)

matches = sim_results.copy()
teams = pd.unique(pd.concat([matches['home_team_name'], matches['away_team_name']]))

team_to_idx = {t: i for i, t in enumerate(teams)}
n_teams = len(teams)
n_matches = len(matches)

print(f" Running {N_SEASON_SIM} season simulations on {DEVICE}...")

# Convert match data into GPU tensors
home_idx = torch.tensor(matches['home_team_name'].map(team_to_idx).values, device=DEVICE)
away_idx = torch.tensor(matches['away_team_name'].map(team_to_idx).values, device=DEVICE)

prob_home = torch.tensor(matches['prob_home_win'].values, device=DEVICE)
prob_draw = torch.tensor(matches['prob_draw'].values, device=DEVICE)
prob_away = torch.tensor(matches['prob_away_win'].values, device=DEVICE)

exp_home_goals = torch.tensor(matches['exp_home_goals'].values, device=DEVICE)
exp_away_goals = torch.tensor(matches['exp_away_goals'].values, device=DEVICE)

# Srorage
season_points_all = {t: [] for t in teams}
season_positions_all = {t: [] for t in teams}

# Precompute categorical distributions for match outcomes
probs = torch.stack([prob_home, prob_draw, prob_away], dim=1)

for sim in range(N_SEASON_SIM):

    # Empty table per simulation (GPU tensor)
    points = torch.zeros(n_teams, device=DEVICE)
    gf = torch.zeros(n_teams, device=DEVICE)
    ga = torch.zeros(n_teams, device=DEVICE)

    # Draw match outcomes
    outcomes = torch.multinomial(probs, 1).squeeze(1)  
    # 0 = Home Win, 1 = Draw, 2 = Away Win

    # Draw Poisson goals 
    gh = torch.poisson(exp_home_goals).float()
    ga_ = torch.poisson(exp_away_goals).float()


    # Update goals
    gf.index_add_(0, home_idx, gh)
    ga.index_add_(0, home_idx, ga_)
    gf.index_add_(0, away_idx, ga_)
    ga.index_add_(0, away_idx, gh)

    # Update points
    # Home win
    mask = (outcomes == 0)
    points.index_add_(0, home_idx[mask], torch.ones(mask.sum(), device=DEVICE) * 3)

    # Away win
    mask = (outcomes == 2)
    points.index_add_(0, away_idx[mask], torch.ones(mask.sum(), device=DEVICE) * 3)

    # Draw
    mask = (outcomes == 1)
    points.index_add_(0, home_idx[mask], torch.ones(mask.sum(), device=DEVICE))
    points.index_add_(0, away_idx[mask], torch.ones(mask.sum(), device=DEVICE))

    # Ranking 
    goal_diff = gf - ga
    table = torch.stack([points, goal_diff, gf], dim=1)

    # Sort by (points desc, goal_diff desc, goals_for desc)
    _, order = torch.sort(table, dim=0, descending=True)
    order = order[:, 0]  # only sort by first key (points), PyTorch keeps tie order stable

    # Record positions and points
    for pos, team_idx in enumerate(order.tolist()):
        team = teams[team_idx]
        season_positions_all[team].append(pos + 1)
        season_points_all[team].append(points[team_idx].item())

print("  Monte Carlo Season Simulation Completed!")
summary = pd.DataFrame({'team': teams})

summary['avg_points'] = summary['team'].apply(lambda t: np.mean(season_points_all[t]))
summary['std_points'] = summary['team'].apply(lambda t: np.std(season_points_all[t]))

summary['prob_win_league'] = summary['team'].apply(lambda t: 
    np.mean(np.array(season_positions_all[t]) == 1)
)
"""  """
summary['prob_top4'] = summary['team'].apply(lambda t: 
    np.mean(np.array(season_positions_all[t]) <= 4)
)

summary['prob_relegation'] = summary['team'].apply(lambda t: 
    np.mean(np.array(season_positions_all[t]) >= len(teams)-2)
)

summary['avg_position'] = summary['team'].apply(lambda t: 
    np.mean(season_positions_all[t])
)

summary = summary.sort_values('avg_position')

summary.to_csv("season_simulation_distributions.csv", index=False)


Fold 1 → RMSE: 0.935, R²: 0.477, MatchAcc: 68.29%
Fold 2 → RMSE: 0.893, R²: 0.525, MatchAcc: 62.93%
Fold 3 → RMSE: 0.845, R²: 0.584, MatchAcc: 65.37%
Fold 4 → RMSE: 0.824, R²: 0.605, MatchAcc: 65.85%
Fold 5 → RMSE: 0.751, R²: 0.552, MatchAcc: 66.83%

Average CV → RMSE: 0.850, R²: 0.549, Accuracy: 65.85%
 Calibrator trained.
 Final models saved.
✅ Simulation saved.
 Running 10000 season simulations on cpu...
  Monte Carlo Season Simulation Completed!


In [3]:
import numpy as np
from sklearn.metrics import r2_score

# Predictions
dtest = xgb.DMatrix(X_test)
y_pred_home = home_final.predict(dtest).clip(0.05, 10)
y_pred_away = away_final.predict(dtest).clip(0.05, 10)

# True values
y_true_home = test_df['home_goals'].values
y_true_away = test_df['away_goals'].values

# Mask rows without NaNs
mask = (~np.isnan(y_true_home)) & (~np.isnan(y_true_away)) & (~np.isnan(y_pred_home)) & (~np.isnan(y_pred_away))

y_true_home = y_true_home[mask]
y_true_away = y_true_away[mask]
y_pred_home = y_pred_home[mask]
y_pred_away = y_pred_away[mask]

# RMSE using np.sqrt
rmse_home = np.sqrt(np.mean((y_true_home - y_pred_home)**2))
rmse_away = np.sqrt(np.mean((y_true_away - y_pred_away)**2))

# R²
r2_home = r2_score(y_true_home, y_pred_home)
r2_away = r2_score(y_true_away, y_pred_away)

# Match outcome accuracy
pred_outcome = np.sign(y_pred_home - y_pred_away)
true_outcome = np.sign(y_true_home - y_true_away)
match_acc = np.mean(pred_outcome == true_outcome)

print(f" Test Set Evaluation:")
print(f"Home RMSE: {rmse_home:.3f}, Away RMSE: {rmse_away:.3f}")
print(f"Home R²: {r2_home:.3f}, Away R²: {r2_away:.3f}")
print(f"Match Outcome Accuracy: {match_acc*100:.2f}%")


 Test Set Evaluation:
Home RMSE: 0.673, Away RMSE: 0.671
Home R²: 0.649, Away R²: 0.196
Match Outcome Accuracy: 80.00%
